# Getting started with PySpark
#### Create spark context and other python imports

In [1]:
%matplotlib inline
from __future__ import print_function, division
from pyspark import SparkConf
from pyspark import SparkContext 
from StringIO import StringIO
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint

conf = SparkConf()
conf.set('spark.executor.instances', 2)
sc = SparkContext()

## A few preliminaries

#### <code>lambda</code> expressions
It is helpful to know lambda expressions and about unpacking arguments arguments for them.

<code>lambda</code> is a statement for an expression that can have a closure.  It's like a function or other callable, but in one line without a return statement.

<code>lambda</code> is useful in pyspark because there is a need for a lot of simple callables.

<code>c=lambda (key,(x,y)): (key, x * y)</code> 

makes c a callable, that can be used like this:

<code>c(('key1', (3, 4)))</code> 

to return 
<code>('key1', 12)</code> 

#### mapping / reducing
In pyspark and other parallel computing frameworks, there is a concept of 'mapping'.

E.g. mapping across a list of tuples and summing the second element of each tuple
```python
x = [(1, 2), (2, 3)]

sum(map(lambda x:x[1], x)) # answer is 5 in python
# or
sc.parallelize(x).map(lambda x:x[1]).sum() # answer is 5 in pyspark
```
In mapping and reducing with pyspark it can be helpful to unpack the arguments to a lambda used in a mapper.

```python
sc.parallelize(x).map(lambda (key, value): value).sum() # 5
```
(x was mapped as a Resilient Distributed Dataset, the key-values were split and the values were summed.

## Common pyspark usage patterns

#### <code>sc.parallelize</code>: create an RDD from a python iterable

<code>sc.parallelize</code> can be called on a list, tuple, generator, or other iterator.  Generators are useful because the full set of elements of the RDD never has to be held in memory in the python main process.

In [87]:
import random
def generate_rands(length=100):
    for idx in range(length):
        yield (random.uniform(0,1), random.uniform(0,1))
rand_rdd = sc.parallelize(generate_rands())

#### We have just created rand_rdd in parallel java processes for use with pyspark map reduce
If we want to get the RDD back to python, we can call <code>.collect()</code>

In [88]:
as_list_again = rand_rdd.collect()
as_list_again[:10]

[(0.27271073923856215, 0.48462588409257845),
 (0.5041987597307358, 0.21326114698680643),
 (0.06855797326697666, 0.36280469242372837),
 (0.46180191158580064, 0.5290749203785821),
 (0.5235684169384603, 0.6409866667779996),
 (0.9965491658529141, 0.7738958760578801),
 (0.8974899207455208, 0.09277112370578644),
 (0.9368523305482879, 0.1568792861957129),
 (0.7556326493761509, 0.7974255788839968),
 (0.641960168786479, 0.06633839325333935)]

#### But a better way to do it is to <code>take</code> a few elements rather than the full RDD
This avoids pulling the entire RDD into the main python process, which breaks parallelism.

In [89]:
rand_rdd.take(5)

[(0.27271073923856215, 0.48462588409257845),
 (0.5041987597307358, 0.21326114698680643),
 (0.06855797326697666, 0.36280469242372837),
 (0.46180191158580064, 0.5290749203785821),
 (0.5235684169384603, 0.6409866667779996)]

#### <code>sc.binaryFiles</code>: read binary HDFS files into an RDD, matching a wildcard pattern

In [112]:
img_files = sc.binaryFiles('/imgs/malestaff*')  # wildcard hadoop distributed file system name
key, img = img_files.take(1)[0] # Don't call .collect() on this one!
pprint((key, img[:100])) # abbreviated the image blob to 100 characters

(u'hdfs://ip-10-95-184-251:9000/imgs/malestaffanonym1anonym1.11.jpg',
 '\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c')


#### <code>sc.pickleFile</code>: read results typically from pyspark that were output with <code>saveAsPickleFile</code>

In pyspark, an RDD may be persisted to the hadoop distributed file system (HDFS) with

<code>x.saveAsPickleFile(some_path)</code> where <code>x</code> is the RDD.

In [10]:
image_measurements = sc.pickleFile('hdfs:///t1/map_each_image/measures')
image_measurements.take(1)  # Don't call .collect() on this one!

[(u'hdfs://ip-10-95-184-251:9000/imgs/femaleekavazekavaz.8.jpg',
  {'cen': array([ 0.56101131,  0.48763534,  0.62458098,  0.15109672,  0.14877951,
           0.11333738,  0.32142827,  0.39909598,  0.16497262,  0.91180402,
           0.76153463,  0.8768959 ,  0.38311961,  0.33375591,  0.34756392,
           0.67436421,  0.59022844,  0.8177439 ,  0.38147092,  0.59025067,
           0.2465435 ,  0.4640739 ,  0.41011605,  0.504408  ,  0.75175792,
           0.2966879 ,  0.27923897,  0.2744056 ,  0.22422786,  0.22581731,
           0.06852739,  0.07509741,  0.04723817,  0.78680694,  0.6175344 ,
           0.65876627], dtype=float32),
   'histo': array([ 0.06236181,  0.12990619,  0.2634635 ,  0.37532961,  0.45033172,
           0.58264619,  0.82682979,  0.07046568,  0.12239435,  0.2413772 ,
           0.43876353,  0.59076869,  0.61009002,  0.66155255,  0.03581263,
           0.09366337,  0.15713109,  0.24893087,  0.43752325,  0.60076404,
           0.81510395], dtype=float32),
   'id': u'hdf

#### <code>.map</code> applys a function to each element of the RDD
This functions maps x and y random numbers and calculates their squares and product

In [12]:
products = rand_rdd.map(lambda (x, y): (x**2, y**2, x*y))
products.take(5)

[(0.03128190306428986, 0.9396735739612871, 0.17144905264460283),
 (0.010824814686933127, 0.2787668909810963, 0.05493268549527555),
 (0.4935927460486683, 0.000175413773665647, 0.009304996842470516),
 (0.09170121975354902, 0.05875481311041671, 0.07340223449335194),
 (0.08568203237814921, 0.1887070298807217, 0.12715660362019884)]

#### <code>.reduce</code> applys an aggregation to an RDD
<code>.reduce</code> takes a callable which is called with two elements of the RDD (repeatedly).  The function must be idempotent, that is it must give the same answer if called more than once on the same inputs.  Summing is an example of an idempotent operation.

Here we calculate the correlation coefficient between the two columns by summing <code class="python">x\*\*2</code>, <code class="python">y\*\*2</code> and <code>x * y</code>

In [153]:
x_variance, y_variance, covar = products.reduce(lambda a, b: [ai + bi for ai, bi in zip(a,b)]) 
# in the above line a and b are both tuples 3 long (x square, y square, x * y)
correlation = covar / (y_variance * x_variance)
correlation

0.021891706457889955

#### <code>.flatMap</code> works like <code>.map</code> but <code>.flatMap</code> returns a list

It is useful for making more elements out of an element of an RDD, like this word count example.

In [155]:
documents = sc.parallelize(['useful for RDDs', 
                            'RDDs are useful', 
                            'flatMap is useful', 
                            'map/reduce is also useful'])
words = documents.flatMap(
    lambda x: [(word, 1) for word in x.split()]  #(from documents rdd to words rdd with 1 for counting)
)
print('words RDD')
pprint(words.collect())
print('Word counts')
words.reduceByKey(
    lambda a, b: a + b  # the word is not passed in, only the count which may be 1 or more when called
).sortBy(
    lambda (word, count): -count   # sort by negative count for descending
).collect()

words RDD
[('useful', 1),
 ('for', 1),
 ('RDDs', 1),
 ('RDDs', 1),
 ('are', 1),
 ('useful', 1),
 ('flatMap', 1),
 ('is', 1),
 ('useful', 1),
 ('map/reduce', 1),
 ('is', 1),
 ('also', 1),
 ('useful', 1)]
Word counts


[('useful', 4),
 ('RDDs', 2),
 ('is', 2),
 ('map/reduce', 1),
 ('are', 1),
 ('also', 1),
 ('flatMap', 1),
 ('for', 1)]

#### Inspecting an RDD, to see what it looks like, we have to call <code>take</code> to see elements of the RDD and we can do type inspection to figure out what is going on.

In [144]:
example = documents.take(1)[0] # take gives a list and we want 1st element
print('example is of type', type(example))

example is of type <type 'str'>


#### <code>.count()</code> gives the number of elements in an RDD

In [125]:
documents.count()

3

## Going through some map reduce ideas with the image files

#### Mapping the <code>load_image</code> function to filenames will load each filename's image into an RDD
The RDD consists of <code>(filename, image_numpy_3d_array)</code> tuples

In [118]:
def load_image(image):
    """Load one image, where image = (key, blob)"""
    from StringIO import StringIO
    from PIL import Image
    img = Image.open(StringIO(image[1]))
    return  image[0], np.asarray(img, dtype=np.uint8)

img_files = sc.binaryFiles('/imgs/malestaff*')  # wildcard hadoop distributed file system name
img_mapped = img_files.map(load_image, img_files)
key, img_3d = img_mapped.take(1)[0] # Don't call .collect() on this one
print(key, "has a shape of ", img_3d.shape)

hdfs://ip-10-95-184-251:9000/imgs/malestaffanonym1anonym1.11.jpg has a shape of  (200, 180, 3)


#### At this point <code>img_mapped</code> is a iterable of tuples (filename, image).  We can in turn map and reduce this RDD to create other RDDs.  Here we are calculating the percentiles in each color band for each image and printing out the first 5 images' percentils as a sample.

In [156]:
labels_bands = zip(('red','green','blue'), range(3))
for label, band in labels_bands:
    percentiles = img_mapped.map(
                        lambda (fname, img): np.percentile(img[:,:,band], (5, 25, 50, 75, 95))
                    )
    print('Percentiles for', label)
    pprint(percentiles.take(5))


Percentiles for red
[array([  11.,   51.,   97.,  122.,  255.]),
 array([  12.,   51.,   98.,  122.,  255.]),
 array([  11.,   49.,   97.,  122.,  255.]),
 array([  11.,   44.,   63.,   75.,  250.]),
 array([  11.,   44.,   63.,   74.,  250.])]
Percentiles for green
[array([  13.,   47.,  127.,  152.,  208.]),
 array([  13.,   47.,  127.,  152.,  208.]),
 array([  13.,   45.,  127.,  152.,  208.]),
 array([  12.,   52.,   85.,   97.,  171.]),
 array([  12.,   53.,   88.,   97.,  169.])]
Percentiles for blue
[array([  12.,   36.,   57.,   93.,  195.]),
 array([  12.,   37.,   59.,   94.,  197.]),
 array([  12.,   36.,   56.,   91.,  196.]),
 array([  10.,   28.,   39.,   58.,  151.]),
 array([  10.,   29.,   39.,   57.,  151.])]


#### With spark, we can also reduce an RDD.  This will take an average of the red percentiles by summing and dividing by the count.

In [157]:
avgs = percentiles.reduce(
    lambda a, b: a + b  # adding numpy percentiles arrays a and b from 2 images
) / percentiles.count() # diving by the number of images
print(avgs)

[  16.46969697   37.20454545   54.87878788   82.80492424  156.34924242]


####Example of groupby operations, with grouping by mean color bins
Below <code>int(np.mean(img[:,:, band]) // 5) * 5</code> creates an integer key that is the pixel 0 to 255 value floored to the nearest 5.  The first map emits this key with the filename.  

In [158]:
for label, band in labels_bands:
    rand_groups = img_mapped.map(
                lambda (fname, img): (int(np.mean(img[:,:, band]) // 5) * 5, fname)
            ).groupByKey(
            ).map(
                lambda (group, fnames): (group, list(fnames))
            ).sortByKey(
            )
    print('Grouping by', label)
    pprint(rand_groups.collect())



Grouping by red
[(45,
  [u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.1.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.13.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.14.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.15.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.19.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.20.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.3.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.5.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.6.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.7.jpg']),
 (50, [u'hdfs://ip-10-95-184-251:9000/imgs/malestaffdorajdoraj.4.jpg']),
 (55,
  [u'hdfs://ip-10-95-184-251:9000/imgs/malestaffdorajdoraj.17.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffdorajdoraj.6.jpg',
   u'hdfs://ip-10-95-184-251:9000/imgs/malestaffdorajdoraj.9.jpg'])

#### The next few steps load the configuration yaml file for image-analyzer, set an output hdfs location, and apply the map_each_image function to write an RDD of image measurements to HDFS

In [126]:
import yaml
from map_each_image import map_each_image
config = yaml.load(open('config.yaml').read())
config['random_state'] = 42
# make a new filename each time for the demo starting with /t1/map_each_image/ 
output_hdfs = '/t1/map_each_image/measures_8'  
input_file_spec = '/imgs/malestaff*'
measures = map_each_image(sc, config, input_file_spec, output_hdfs)
measures

PythonRDD[419] at RDD at PythonRDD.scala:43

#### later load the image measurements from HDFS using unpickling

In [127]:
measures = sc.pickleFile(output_hdfs)
measures.take(1) # don't call .collect() here

[(u'hdfs://ip-10-95-184-251:9000/imgs/malestaffanonym1anonym1.11.jpg',
  {'cen': array([ 0.45033672,  0.63002706,  0.24756569,  0.10609439,  0.10652294,
           0.10405473,  0.98829114,  0.81236833,  0.7515012 ,  0.31112835,
           0.28374338,  0.26491776,  0.37117678,  0.50180066,  0.15738003,
           0.95043582,  0.63999772,  0.60735929,  0.46374711,  0.3828384 ,
           0.3455241 ,  0.53057158,  0.639386  ,  0.84825116,  0.38541538,
           0.55300885,  0.22826292,  0.65902579,  0.50548172,  0.50747198,
           0.04695916,  0.05420781,  0.0555218 ,  0.20017309,  0.19093239,
           0.18037727], dtype=float32),
   'histo': array([ 0.04643805,  0.09260353,  0.19718982,  0.3818205 ,  0.47212642,
           0.69891268,  0.99839503,  0.05481956,  0.09406018,  0.18146341,
           0.49939993,  0.59083205,  0.64937669,  0.81560934,  0.04943289,
           0.0914389 ,  0.1412462 ,  0.22058824,  0.35655639,  0.57910204,
           0.76368791], dtype=float32),
   'id':

#### From the image analyzer code we know that the <code>'histo'</code> key is the 3 band histogram flattened into one array, so we can check its size and know where the median colors are.  For example, if it is 21 elements long, then the first 7 are the red histogram and the middle of those 7 elements is the median red color.

In [128]:
def get_median_from_histo(histo, band):
    lh = len(histo) // 3
    return histo[lh // 2 + 1 + band * lh]

#### As an exercise, sort the images by median color in each band, take the 5 image file names that are lowest in that color

In [130]:
for label, band in labels_bands:
    sorted_filenames = measures.sortBy(
        lambda (key, value): get_median_from_histo(value['histo'], band)
    ).map(
        lambda (key, value): key
    ).collect()
    print('Lowest 5 images in %s ' % label)
    pprint(sorted_filenames[:5])


Lowest 5 images in red 
[u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.3.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.6.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.20.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.17.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.15.jpg']
Lowest 5 images in green 
[u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.20.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.5.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.19.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.3.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.1.jpg']
Lowest 5 images in blue 
[u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.6.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.3.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffmartinmartin.14.jpg',
 u'hdfs://ip-10-95-184-251:9000/imgs/malestaffma

810

####Make a function to print out images in the same order they are found in <code>measures</code>.

#### Joins in PySpark work on RDDs, using equality on the first item in tuples to determine the join

Here is are several join examples with a 2 lists of tuples that are spark RDDs from python lists

In [131]:
list_a = sc.parallelize([(1, 1), (3, 3), (5, 10)]) # a has some keys not in b's keys
list_b = sc.parallelize([(1, 2), (3, 4), (7, 12)]) # b has some keys not in a's keys
list_a.join(list_b).collect()

[(1, (1, 2)), (3, (3, 4))]

In [132]:
list_b.join(list_a).collect()

[(1, (2, 1)), (3, (4, 3))]

In [133]:
list_a.rightOuterJoin(list_b).collect()

[(1, (1, 2)), (3, (3, 4)), (7, (None, 12))]

In [134]:
list_b.leftOuterJoin(list_a).collect()

[(1, (2, 1)), (3, (4, 3)), (7, (12, None))]

In [135]:
list_b.rightOuterJoin(list_a).collect()

[(1, (2, 1)), (3, (4, 3)), (5, (None, 10))]

In [136]:
list_a.leftOuterJoin(list_b).collect()

[(1, (1, 2)), (3, (3, 4)), (5, (10, None))]

In [137]:
list_a.fullOuterJoin(list_b).collect()

[(1, (1, 2)), (3, (3, 4)), (5, (10, None)), (7, (None, 12))]

#### Joins with the image data
In the example images we have <code>hdfs:///imgs/</code> which includes the original and <code>hdfs:///fuzzy/</code> which is a fuzzy version of those.  We can map the measurements of fuzzy and original images and join on ward cluster hashes.  In each image's measures dictionary, the ward clusters are saved in a list, as shown above.  We can use <code>flatMap</code> to flatten that list and create an RDD with tuples of <code>(one_ward_cluster_hash, image_file_name)</code>.

In [138]:
candidates = sc.pickleFile('hdfs:///t1/candidates/c1/measures')

In [139]:
def flatten_ward(rdd):
    return rdd.flatMap(lambda (key, value): [(wc, key) for wc in value['ward']])

In [141]:
candidates_flat = flatten_ward(candidates)
originals_flat = flatten_ward(measures)
pprint(candidates_flat.take(1))
pprint(originals_flat.take(1))

[(3031575136247091488,
  u'hdfs://ip-10-95-184-251:9000/fuzzy/femaleekavazekavaz.8.jpg')]
[(4390753332276699091,
  u'hdfs://ip-10-95-184-251:9000/imgs/malestaffanonym1anonym1.11.jpg')]


In [142]:
joined = candidates_flat.join(originals_flat)

In [143]:
joined.groupByKey().map(lambda (key, value): (key, list(value))).take(4)

[(-1842252707656104988,
  [(u'hdfs://ip-10-95-184-251:9000/fuzzy/femaleekavazekavaz.8.jpg',
    u'hdfs://ip-10-95-184-251:9000/imgs/malestaffanonym1anonym1.11.jpg'),
   (u'hdfs://ip-10-95-184-251:9000/fuzzy/femaleekavazekavaz.8.jpg',
    u'hdfs://ip-10-95-184-251:9000/imgs/malestaffcwangcwang.20.jpg'),
   (u'hdfs://ip-10-95-184-251:9000/fuzzy/femaleekavazekavaz.8.jpg',
    u'hdfs://ip-10-95-184-251:9000/imgs/malestaffdorajdoraj.13.jpg'),
   (u'hdfs://ip-10-95-184-251:9000/fuzzy/femaleekavazekavaz.8.jpg',
    u'hdfs://ip-10-95-184-251:9000/imgs/malestaffdorajdoraj.6.jpg'),
   (u'hdfs://ip-10-95-184-251:9000/fuzzy/femaleekavazekavaz.8.jpg',
    u'hdfs://ip-10-95-184-251:9000/imgs/malestaffdorajdoraj.9.jpg'),
   (u'hdfs://ip-10-95-184-251:9000/fuzzy/femalembutlembutle.19.jpg',
    u'hdfs://ip-10-95-184-251:9000/imgs/malestaffanonym1anonym1.11.jpg'),
   (u'hdfs://ip-10-95-184-251:9000/fuzzy/femalembutlembutle.19.jpg',
    u'hdfs://ip-10-95-184-251:9000/imgs/malestaffcwangcwang.20.jpg'),
  

### Explore_Spark_Results.ipynb in this directory goes further into the image example